<a href="https://colab.research.google.com/github/Macielyoung/Colab_Repo/blob/master/Copy_of_Train_a_GPT_2_Text_Generating_Model_w_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  免费用 Google GPU 来 finetune GPT-2

英文来自： [Max Woolf](http://minimaxir.com)

中文修改：Andy

最近 GPT2 放出了 345M 的中型模型（之前是 117M 模型），于是又引起一波 Finetune 大潮，于是自己也就顺便抽出时间拿这个 345M 模型 finetune 了几个玩。

这部分主要是教大家用 `gpt-2-simple` 库，并且蹭 Google Colab 免费 GPU 来对 GPT2 模型进行 finetune，这篇 Notebook 业主要来自这个库的作者。


 `gpt-2-simple` 库 github 地址：https://github.com/minimaxir/gpt-2-simple



#### 开始前准备

1. 首先，要想薅 Google 羊毛，就得现有个 Google 账号，[注册](https://accounts.google.com/signup);
2. 然后把这个 Notebook 保存到自己 Google Drive 里去，便于运行和修改。(File -> Save a Copy in Drive)
3. 之后，同样的既然要薅 Google 羊毛，就还得用他家产品，所以保证用 Chrome 浏览器；
4. 最后，安装环境和 import 各种需要包，运行下面的 Cell 就好了:

In [0]:
!pip install -q gpt_2_simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

## 查看 GPU

既然要薅羊毛，那就要薅个清楚，先来看看 GPU 信息吧。

现在 Colab 更新过一次，已经再使用英伟达 T4 GPU了，比之前的 K80 要快些，而且内存也要大些，有 16 G。这么大的内存 finetune 的时候拿大模型来也都没问题了，而且可以生成更多文本。


In [0]:
!nvidia-smi

Wed May 15 12:28:39 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    15W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## 下载 GPT-2 模型

首先，finetune 的第一步，先把预训练模型下载下来，就像做菜要先备好材料。

正如开头提到的，现在放出了两个 GPT-2 模型， 
* `117M` : 小模型，大概有 500MB 大小；
* `345M`: 最近放出的中型模型，1.5 GB.

之后 finetune 的时候，只需要修改模型名字 `model_name` 参数就可以选择模型。大模型有更多的知识，但同时也要更长时间来进行 finetune 和生成。

下面的 cell 就是用来下载预训练模型文件的，会保持在 Colab 的 `/models/<model_name>`下，点左边栏上面的 `Files` 也可以看到。

模型不是永久保存的，下次 finetune 的时候还要重新下载。

In [0]:
gpt2.download_gpt2(model_name="345M")

Fetching checkpoint: 1.00kit [00:00, 641kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 48.7Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 620kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:26, 53.9Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 2.89Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 46.6Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 36.6Mit/s]                                                       


## 装载 Google Drive （类似云盘）

想输入训练数据，然后保存模型的最好方法，就是把 Google Drive 给装载进来，之后把训练数据放在上面，保存也直接放上面


下面这个 cell 就是装载个人 Google Driver 的命令。会出现一个链接和提示框，点击链接获得授权码，粘贴到提示框里去。

In [0]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 上传训练文本

也是在左边的侧栏里面，直接点击`Files` 里面的上传 `UPLOAD` 就好了。

![alt text](https://i.imgur.com/TGcZT4h.png)

之后也可以传任意自己想 finetune 的文本，这里我们用老友记十季的剧本来作为训练数据。直接上传的话推荐小于 10M 的数据。


In [0]:
file_name = "friends.txt"

如果大于 10M 的文本，那就推荐先上传到 google drive (根目录) 里然后再拷过来。如果报错，再重新运行装载命令。

In [0]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

于是准备好模型和数据，就要开始今天的主菜了，finetune GPT2. 

下面的代码会运行一个 Tensorflow Sessioin，读取训练设定，然后按照设定训练。`steps` 用来指定 finetune 多少步，如果设成 -1 的话就表示不停 finetune 下去。

模型文件会保存在`/checkpoint/run1` 里面，会根据`save_every`来保存一定步数文件，结束训练的时候也会保存。

训练结束需要大概半个小时，一定要记得保存模型。



**重要提示**：如果想要重复运行下面 cell 的话，需要重启服务，`Runtime/Restart Runtime`.

gpt2.finetune 训练参数介绍：


*  **`restore_from`**:  `fresh` 是指从 GPT2 原模型开始, 而 `latest`是从之前 finetune 保存的模型继续训练.
* **`sample_every`**: 每多少步输出样本，看看训练效果
* **`print_every`**: 每多少步打印训练的一些参数，从左到右，步数、时间，loss，平均loss
* **`learning_rate`**: 学习率 (默认 `1e-4`, 如果数据小于1MB的话可以调低到 `1e-5`)
*  **`run_name`**: 运行的时候，保存模型到`checkpoint`下哪个子文件，默认 `run1`

In [0]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='345M',
              steps=1000,
              restore_from='fresh',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.random.categorical instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Loading checkpoint models/345M/model.ckpt
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/345M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:07<00:00,  7.86s/it]


dataset has 1464638 tokens
Training...
[10 | 23.14] loss=2.54 avg=2.54
[20 | 38.13] loss=2.74 avg=2.64
[30 | 53.24] loss=2.31 avg=2.53
[40 | 68.47] loss=2.62 avg=2.55
[50 | 83.82] loss=2.55 avg=2.55
[60 | 99.25] loss=2.29 avg=2.51
[70 | 114.81] loss=2.28 avg=2.47
[80 | 130.46] loss=2.31 avg=2.45
[90 | 146.22] loss=2.56 avg=2.46
[100 | 162.06] loss=2.74 avg=2.49
[110 | 177.97] loss=2.46 avg=2.49
[120 | 193.95] loss=2.13 avg=2.46
[130 | 209.99] loss=2.58 avg=2.47
[140 | 226.06] loss=2.43 avg=2.47
[150 | 242.11] loss=2.49 avg=2.47
[160 | 258.17] loss=2.75 avg=2.49
[170 | 274.26] loss=2.16 avg=2.47
[180 | 290.37] loss=2.50 avg=2.47
[190 | 306.50] loss=1.98 avg=2.44
[200 | 322.64] loss=2.31 avg=2.43
======== SAMPLE 1 ========

Skipper: But I don''t think it was that bad!
Joey: Hey, how are we today, man?
Chandler: Hey, what are you doing?
Joey: I don’t think this is so bad.
Chandler: You know what? Whatever you are doing, just tell me. (To Joey) Tell it!
Joey: Okay. (Hands him a cigar.)
[Sc

模型训练完之后，你就可以把保存模型，直接拷到 Google Drive 下面去了。


之后如果想在自己本地使用的话，就直接从 Google Drive 直接下载就可以了。

In [0]:
gpt2.copy_checkpoint_to_gdrive()

OK，搞定了。

现在就尽情调戏自己训练好的模型吧。

## 载入训练好的模型

下面的 cell 会把你 google drive 下的 `checkpoint` 拷进来。

如果是一口气直接训练过来的话，就不用运行这一步了。

In [0]:
gpt2.copy_checkpoint_from_gdrive()

FileExistsError: ignored

之后就是，从模型中载入训练好的模型。

这个 cell 和之前训练 cell 一样，如果想重复跑的话得重启环境。


In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess)

Instructions for updating:
Colocations handled automatically by placer.
Loading checkpoint checkpoint/run1/model-1000
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


## 从训练好模型生成文本

载入模型之后，就是激动人心的生成部分了！

下面这个是无条件（Unconditional）的生成，也就是没有任何限制，让模型自己生成。

结果并不理想，可能因为 finetune 的步数太小了，还有一些生成参数没有设置，这里只是个示例所以就只设 1000 步训练，想要更好效果可以设更大一些。

In [0]:
gpt2.generate(sess)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.random.categorical instead.
Actress - The One With The Wedding Dress

Ally: (entering) Hey!
Ross: Hi!
Rachel: Hi!
Phoebe: Hey! (they hug)
Ross: Hi!
Rachel: Hi! I didn't know you were getting married!
Phoebe: You didn't know.
Ross: I didn't know!
Rachel: I didn't know but y'know, I have to get married!
Phoebe: Oh, I just...
Rachel: I don't know.
Phoebe: Okay, well then, I have to get married.
Rachel: Oh, oh, I can't wait to get married! You know, if you don't know I'm gonna talk to you. I mean, I know that you're gay, but... I mean, you're gay and-and you're married.
Phoebe: Oh, I'm so sorry, but y'know I just have to go to the wedding.
Rachel: Oh no, no, no, you don't have to go to the wedding.
Phoebe: No, I'll see you tomorrow.
Rachel: Okay. (to Ross) Good for you!
Ross: Great. Thank you!
Rachel: Thank you!
Ross: Okay. (to Phoebe) No, no, I'm not gay. I'm, I'm just, I'm just going to see you tomorrow.
Ph

之后如果你想要创建 API 获得生成的文本的话，直接运行 `text = gpt2.generate(sess, return_as_list=True)[0]` 就好了。

然后，上面提到了无条件生成，那有条件生成(Conditional)呢，我们可以直接用`prefix`来指定一段文本，然后让模型来续写。

此外，还可以同时生成几个样本，通过指定`nsamples`。还有，通过设定`batch_size`还可以加速生成过程。


`gpt2.generate` 和相关函数其他一些有用的选项：

*  **`length`**: 生成文本长度 (默认 1023, 也是可设最大长度)
* **`temperature`**: temperature 越高，生成的就随意。 (默认 0.7，推荐 0.7 到 1.0之间)
* **`top_k`**: 将输出限定在 top k 里面 (默认0，也就是不使用。推荐在生成效果差的时候使用，可以设top_k=40)
* **`truncate`**: 从指定符号阶段生成文本 (比如设 `truncate='<|endoftext|>'`, 那么就会取第一个'<|endoftext|>'前的文本作为输出). 可以和一个比较小的`length`值搭配使用.
*  **`include_prefix`**: 如果用了 `truncate` 和 `include_prefix=False`, 那么在返回文本中就不会包含`prefix`里的文本。

In [0]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="Rachel: Andy, marry me!",
              nsamples=5,
              batch_size=5,
              top_k=40
              )

R: Andy, marry me!
Andy: (hugs her) Well, it's a good one, no.
Rachel: Oh-oh, I’m sorry.
Andy: I’ve just got to get a new apartment.
Rachel: Oh no, no-no!!
Andy: No-no, no. No, no-no.
Rachel: No-no-no.
Andy: No, no, no-no-no. (Moves Rachel to the door.) (opens it) You’re not moving.
Rachel: No-no-no-no, not moving. No!!
Andy: Oh, come on! You’re not moving!
Rachel: No, not moving. No!!
Andy: No-no, not moving. No!!
Rachel: No-no, not moving. No!!
Andy: No-no, not moving. No!!
Rachel: No, not moving!!
Andy: No-no, not moving. No!!
Rachel: No-no!
Andy: No-no-no!!
Rachel: (closes the door and runs out on her own, slamming the door behind her) I’m

R: Andy, marry me!
Joey: No! No! No!
Chandler: You can't do this!
Rachel: We have to do something!
Joey: No! You can't!
Chandler: Yes you can!
Joey: Yes you can!
Chandler: I can!
Joey: Yeah! (He pushes Chandler back onto his bed.)
Chandler: I can!
Joey: Okay! Okay! Okay! Okay- Okay!
Chandler: Okay! Okay!
Joey: Okay! Okay! (He gets up to get up.)

如果想要生成大量文本的话，就可以用下面 cell 的命令。

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

files.download(gen_file)

# 额外提示

如果出现错误 (比如 GPU Sync Fail or out-of-memory/OOM), 可以用下列命令来强行杀掉并重启:

In [0]:
!kill -9 -1

欢迎转载，务必注明出处。


关注公众号：Andy的写作间(andy_writing) ，更多自然语言处理，深度学习，AI，写作等等等